In [1]:
%reload_ext watermark
%watermark -v -p sklearn,os,glob,cv2,shutil,joblib,gc,random,re,tensorflow,numpy,tqdm,matplotlib,pandas,sklearn,datetime,random,warning

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.34.0

sklearn   : 1.0.2
os        : unknown
glob      : unknown
cv2       : 4.6.0
shutil    : unknown
joblib    : 1.1.0
gc        : unknown
random    : unknown
re        : 2.2.1
tensorflow: 2.8.0
numpy     : 1.22.3
tqdm      : 4.63.1
matplotlib: 3.5.1
pandas    : 1.4.1
datetime  : unknown



In [2]:
import os, glob, cv2, shutil, joblib,gc, random, re, multiprocessing
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
import random
import warnings
warnings.filterwarnings(action='ignore')
import datetime
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import sklearn
from sklearn.ensemble import RandomForestClassifier

print(datetime.datetime.today())

2022-11-29 17:08:19.961051


In [3]:
def video_gray2rgb(df):
    out=np.zeros(df.shape[:3]+(3,),dtype='u1')
    for i in range(df.shape[0]):
        img=cv2.cvtColor(df[i,:,:,0],cv2.COLOR_GRAY2RGB)
        out[i,:,:,:]=img   
        del img
    return out
def batch_avi(data,idx,BATCH_SIZE=1024,font=cv2.FONT_HERSHEY_SIMPLEX):
    def label_vis(data,i,font):
        input_img=cv2.cvtColor(data[i,:,:,0],cv2.COLOR_GRAY2RGB
                            ).astype('u1')
        
        pred_img=cv2.cvtColor(data[i,:,:,1]*255,cv2.COLOR_GRAY2RGB
                            ).astype('u1')
        pred_img[:,:,(0,2)]=0
        
        label_img=cv2.addWeighted(input_img,1,pred_img,.2,0)
        text=f"Index : {i}"
        cv2.putText(label_img,text, (80,30),font,1,(255,0,0),2)
        
        return np.expand_dims(label_img,axis=0)
    return np.concatenate([label_vis(data,i, font) 
                    for i in range((BATCH_SIZE*idx),
                                   (np.min([data.shape[0],(idx+1)*BATCH_SIZE])))])

def write_avi(data,file_name,fps=30):
    fourcc = cv2.VideoWriter_fourcc(*"DIVX")
    (width, height)=data.shape[1:3]
    out = cv2.VideoWriter(file_name, fourcc, fps, (height, width),isColor=True)
    for i in range(len(data)):
        out.write(data[i])
    out.release()

def get_feature(file):
    df=pd.read_csv(file,sep='\t')
    df['diagnosis_group']=file.split('/')[-1].split('.')[0]
    ls=list()
    for i in df.patient_id.values:
        ls.append(glob.glob(
            f"/home/data/**/{i}/")[0])
    df['input_path']=ls
    output=df[['patient_id','age','sex','input_path','diagnosis_group']]
    output.loc[:,['LA','LP','Left','RA','RP','Right']]=0
    for i,(path,patent) in tqdm(enumerate(df[['input_path','patient_id']].values)):
        try:
            file=glob.glob(f"{path}/*.tsv")[0]
            file=os.path.realpath(file)
            output.loc[i,'path']=file
            feature_df=pd.read_csv(file,sep='\t')
            feature=feature_df[['Direction','Gain']].groupby(['Direction']).mean().T
            output.loc[i,feature.columns]=feature.values[0,:]
        except:
            output.loc[i,'path']=''
    output=output[['age','sex','diagnosis_group','LA','LP','Left','RA','RP','Right','path']]

    return output
print(datetime.datetime.today())

2022-11-29 17:08:19.971490


# 모델링

In [4]:
files=glob.glob("/home/data/*.tsv")
files=[i for i in files if not re.compile('stats').findall(i)]
print(datetime.datetime.today())

2022-11-29 17:08:19.975588


In [5]:
if 'model_files.csv' not in os.listdir():
    print(datetime.datetime.today())
    num_core=multiprocessing.cpu_count()
    pd.set_option('mode.chained_assignment',  None) 
    with joblib.Parallel(n_jobs=num_core-2) as parallel:
        model_files = parallel(joblib.delayed(get_feature)(i) for i in tqdm(files))
    pd.set_option('mode.chained_assignment', 'warn')
    model_files=pd.concat(model_files)
    model_files.loc[model_files.sex=='F ','sex']='F'
    model_files.loc[model_files.sex=='F ','sex']='F'
    model_files.loc[model_files.sex=='f','sex']='F'
    model_files.loc[model_files.sex=='m','sex']='M'
    model_files[["F","M"]]=pd.get_dummies(model_files['sex'])
    model_files=model_files.drop('sex',axis=1)
    model_files=model_files.dropna()
    model_files=model_files[model_files.path!=''].reset_index().drop('index',axis=1)
    print(datetime.datetime.today())
    model_files.to_csv("model_files.csv",index=False)
else:
    model_files=pd.read_csv('model_files.csv')

2022-11-29 17:08:19.981024


100%|██████████| 3/3 [00:00<00:00, 17452.03it/s]
/tmp/ipykernel_232949/3922528268.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
0it [00:00, ?it/s]/tmp/ipykernel_232949/3922528268.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_232949/3922528268.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

2022-11-29 17:18:48.161766


7123it [10:27, 11.36it/s]

In [6]:
print(model_files.diagnosis_group.value_counts())
print(model_files.diagnosis_group.value_counts()/model_files.diagnosis_group.value_counts().sum())

Normal        6933
Peripheral    6910
Central       2683
Name: diagnosis_group, dtype: int64
Normal        0.419521
Peripheral    0.418129
Central       0.162350
Name: diagnosis_group, dtype: float64


## 모델링 수행

In [7]:
print(datetime.datetime.today())
kf = KFold(n_splits = 3, shuffle = True,random_state=42)
target_label=np.array(['Abnormal','Normal'])
params={"num_leaves":[20,40,60,80,100],
        "min_child_samples":[5,10,15],
        "max_depth":[-1,5,10,2],
        "learning_rate":[0.05,0.1,0.2],
        "reg_alpha":[0,0.01,0.03],
       }

dfs=list()
tp_sum=dict();fn_sum=dict();fp_sum=dict();tn_sum=dict()
for K, (train_idx,test_idx) in enumerate(kf.split(model_files),1):    
    for cutoff in np.arange(0,1.1,0.05):
        for target in target_label:
            cond=f"K_{K}_target_{target}_cutoff_{cutoff:.2f}"
            tp_sum[cond]=0;fn_sum[cond]=0;fp_sum[cond]=0;tn_sum[cond]=0
    
    train_files=model_files.loc[train_idx,:]
    test_files=model_files.loc[test_idx,:]
    tr_x=train_files.drop(['path','diagnosis_group'],axis=1)
    tr_y=(train_files.diagnosis_group=='Normal').astype('u1')
    
    train_files, valid_files=train_test_split(
        train_files, test_size=.2, random_state=42)
    train_files=train_files.sort_index()
    valid_files=valid_files.sort_index()
    test_files=test_files.sort_index()

    tr_x_=train_files.drop(['path','diagnosis_group'],axis=1)
    tr_y_=(train_files.diagnosis_group=='Normal').astype('u1')
    
    val_x_=valid_files.drop(['path','diagnosis_group'],axis=1)
    val_y_=(valid_files.diagnosis_group=='Normal').astype('u1')
    
    te_x=test_files.drop(['path','diagnosis_group'],axis=1)
    te_y=(test_files.diagnosis_group=='Normal').astype('u1')

    print(train_files.diagnosis_group.value_counts())    
    print(test_files.diagnosis_group.value_counts())
    ###################
    class_weight={
        0:.6,1:.4
    }
    
#     lgbm=lgb.LGBMClassifier(
#         n_estimators=400,random_state=10042,class_weight=class_weight)
#     clf=GridSearchCV(lgbm,params,scoring="accuracy",verbose=1,n_jobs=-1,cv=3)
#     clf.fit(X=tr_x,y=tr_y)
#     lgbm.set_params(**clf.best_params_)
#     lgbm.fit(tr_x_, tr_y_, early_stopping_rounds=300,verbose=0,eval_set=[(val_x_,val_y_)])
#     joblib.dump(lgbm,f'lgbm_{K}_fold_v2.pkl')
    
    lgbm=joblib.load(f'lgbm_{K}_fold_v2.pkl')
    y_preds=lgbm.predict_proba(te_x)
    test_files['pred']=np.apply_along_axis(np.argmax,1,y_preds)
    test_files['y_true']=te_y
    test_files['proba']=y_preds[:,1]
    dfs.append(test_files)
pd.concat(dfs).to_csv('진단모델_output_final.csv',index=False)
print(datetime.datetime.today())

2022-11-29 17:18:48.261116
Normal        3741
Peripheral    3656
Central       1416
Name: diagnosis_group, dtype: int64
Peripheral    2322
Normal        2291
Central        896
Name: diagnosis_group, dtype: int64
Normal        3741
Peripheral    3666
Central       1406
Name: diagnosis_group, dtype: int64
Peripheral    2310
Normal        2291
Central        908
Name: diagnosis_group, dtype: int64
Peripheral    3700
Normal        3677
Central       1437
Name: diagnosis_group, dtype: int64
Normal        2351
Peripheral    2278
Central        879
Name: diagnosis_group, dtype: int64
2022-11-29 17:18:48.460890


In [8]:
scores=dict()
for i,df in enumerate(dfs):
    scores.update({i:roc_auc_score(df.y_true, df.proba)})
    
with open("진단모델지표.json", 'w') as outfile:
    json.dump(scores, outfile)

In [9]:
scores

{0: 0.6737553438903113, 1: 0.7043176897520196, 2: 0.6725239881343666}